In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from tqdm import tqdm
import seaborn as sns
from tqdm import tqdm

headers = {  # 헤더 설정
    'Content-Type': 'application/json'  # JSON 형식 설정
}

# url 요청 형식
# https://apihub.kma.go.kr/api/typ01/url/fct_shrt_reg.php? <- 기본적으로 API 요청시 적어주는 주소
# tmfc=0&stn=123134& <- 요청인자, API요청할때 정보 같은거 뭐 가져올지 변수값 할당해주는거, 해당 예시는 tmfc를 0, stn을 123134로 할당한거
# authKey = 자기가 발급받은 기상청 API 키, 회원가입해서 따로 발급받아야함, response 요청 url 맨 마지막에 들어있는 부분 말이야


#단기예보 구역 정보 조회, 기상청 api 활용 : https://apihub.kma.go.kr/
url = 'https://apihub.kma.go.kr/api/typ01/url/fct_shrt_reg.php?tmfc=0&authKey=zxiqblJaSgSYqm5SWioEeA'
response = requests.get(url, headers=headers)  # GET 요청


In [2]:
print(response.text)
# 11H000EB 200308050000 210012310000 E      진주-함양 
# 11H20701 201610131800 210012310000 C      진주
# 광주가 ㅅ팔 1개가 아님 조심
# 11F20500 199001010000 210012310000 B      광주.전남중부내륙 
# 11F20501 199001010000 210012310000 C      광주

#START7777
#--------------------------------------------------------------------------------------------------
#  단기예보구역 조회
#---------------------------------------------------------------------------------------------------
#  1. REG_ID   : 예보구역코드
#  2. TM_ST    : 시작시각(년월일시분,KST)
#  3. TM_ED    : 종료시각(년월일시분,KST)
#  4. REG_SP   : 특성 (A:육상광역,B:육상국지,C:도시,D:산악,E:고속도로,H:해상광역,I:해상국지,J:연안바다,K:해수욕장,L:연안항로,M:먼항로,P:산악)
#  5. REG_NAME : 예보구역명
#---------------------------------------------------------------------------------------------------
# REG_ID TM_ST        TM_ED        REG_SP REG_NAME
11000000 199001010000 210012310000 A      육상 
11A00000 201501221100 210012310000 A      서해5도 
11A00100 199001010000 210012310000 B      서해5도 
11A00101 201610131800 210012310000 C      백령도 
11A00102 199001010000 210012310000 C      연평도 
11A00103 199001010000 210012310000 C      소청도 
11B00000 201101110800 210012310000 A      서울.인천.경기 
11B000D1 200707250000 210012310000 D      북한.도봉산 
11B000D2 200204240000 2100

In [3]:
#단기 육상 기상 예보 조회(말그대로 그당시에 기상 캐스터가 미래에 날씨가 어떻게 될거다 라고 예보한 내용을 가져오는거, 실제 날씨 측정정보 X, 날씨 예측정보 O)


# https://apihub.kma.go.kr/
url = 'https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl.php?reg=11H20701&tmfc1=2013121106&tmfc2=2013121118&disp=0&help=1&authKey=zxiqblJaSgSYqm5SWioEeA'
response = requests.get(url, headers=headers)  # GET 요청

print(response.text)

#START7777
#--------------------------------------------------------------------------------------------------
#  단기예보 육상 조회 [입력인수형태][예] ?reg=&tmfc1=2013121018&tmfc2=2013121106&disp=0&help=1
#--------------------------------------------------------------------------------------------------
#  1. REG_ID   : 예보구역코드
#  2. TM_FC    : 발표시각(년월일시분,KST)
#  3. TM_EF    : 발효시각(년월일시분,KST)
#  4. MOD      : 구간 (A01(24시간),A02(12시간))
#  5. NE       : 발효번호
#  6. STN      : 발표관서
#  7. C        : 발표코드
#  8. MAN_ID   : 예보관ID
#  9. MAN_FC   : 예보관명
# 10. W1       : 풍향1(16방위)
# 11. T        : 풍향경향(1:-, 2:후)
# 12. W2       : 풍향2(16방위)
# 13. TA       : 기온
# 14. ST       : 강수확률(%)
# 15. SKY      : 하늘상태코드 (DB01(맑음),DB02(구름조금),DB03(구름많음),DB04(흐림))
# 16. PREP     : 강수유무코드 (0(없음),1(비),2(비/눈),3(눈),4(눈/비(~'19.6.4.),소나기('19.6.4~)))
# 17. WF       : 예보
#--------------------------------------------------------------------------------------------------
# REG_ID TM_FC        TM_EF        MOD NE STN C MAN_ID       MAN_FC 

In [4]:
#실제 지상 관측 자료 조회, https://apihub.kma.go.kr/#popup1
#stn값이 요청인자로 들어가는데, 위에 단기 육상 기상 예보 API조회로 조회해보면 stn값이 나온다
# 기상청 관측지점정보조회 url : https://minwon.kma.go.kr/main/obvStn.do    , 지점번호가 stn 번호인거 같음,어디서 관측했는지 확실히 확인하기위한 용도
# 실시간 시스템 만드려고 조회하는거 구현한거, 그냥 있는 데이터 통짜 받으려면 https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36들어가면댐
# 근데 저기도 일정 기간별 조회라 노가다 해야함

url = 'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php?tm1=202201010000&tm2=202201312330&stn=192&help=1&authKey=Brv79B9xSp-7-_QfcRqfAw'
response = requests.get(url, headers=headers)  # GET 요청

weather_observation_data = response.text

for i, n in enumerate(weather_observation_data.split('\n')):
    print(i, n)
    if(i>60):
        break


#밑에 # YYMMDDHHMI이런거 나와잇는거 보일거임 52,53번째 줄, 이거 2줄인 이유가 52번째 줄은 컬럼명이고, 53번째는 부가적인 설명이나 단위를 적어놓은것
#아래와 같이 나타낸거임, 예를들어 WD는 16방위라 16이라한거고, WS는 바람속도라서 m/s 이런식으로
        
# YYMMDDHHMI STN  WD   WS GST  GST  GST     PA     PS PT    PR    TA    TD    HM    PV     RN     RN     RN     RN     SD     SD     SD WC WP WW                      CA  CA   CH CT        CT  CT  CT    VS   SS    SI ST    TS    TE    TE    TE    TE  ST   WH BF IR IX
#        KST  ID  16  m/s  WD   WS   TM    hPa    hPa  -   hPa     C     C     %   hPa     mm    DAY    JUN    INT    HR3    DAY    TOT -- -- ---------------------- TOT MID  MIN -------- TOP MID LOW                  GD     C     5    10    20    30 SEA    m --  -  -

0 {
1   "result" : {
2     "status" : 403,
3     "message" : "일일 최대 호출 건수 제한으로 사용할 수 없습니다."
4   }
5 }


In [5]:
# 기간단위가 아닌 특정날짜를 지목해서 데이터를 가져오기, api 제한으로 이해 전체데이터 가져오는건 실패
'''
url = 'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php?tm=202201010000&stn=192&help=0&authKey=Brv79B9xSp-7-_QfcRqfAw'
response = requests.get(url, headers=headers)  # GET 요청
weather_observation_data = response.text

# 컬럼추출
test = weather_observation_data.split('\n')[2]
test = np.array(test.split(' '))
columns = list(test[test != ''])
columns.remove('#')
print(columns)

#데이터 추출
def day28():
    return [f"{i:02}" for i in range(1, 29)]
def day29():
    return [f"{i:02}" for i in range(1, 29)]

def day30():
    return [f"{i:02}" for i in range(1, 31)]
def day31():
    return [f"{i:02}" for i in range(1, 32)]


data = []
year = dict()
year['2022'] = [day31(), day28(), day31(), day30(), day31(), day30(), day31(), day31(), day30(), day31(), day30(), day31()]
year['2023'] = [day31(), day28(), day31(), day30(), day31(), day30(), day31(), day31(), day30(), day31(), day30(), day31()]
year['2024'] = [day31(), day29(), day31(), day30(), day31(), day30(), day31(), day31(), day30(), day31(), day30(), day31()]
month = [f"{i:02}" for i in range(1, 13)]
time = [f"{i:02}00" for i in range(24)]


for y in year.keys():
    for m in month:
        for d_list in tqdm(year[y]):
            for d in d_list:
                for t in time:
                    url = f'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php?tm={y}{m}{d}{t}&stn=192&help=0&authKey=Brv79B9xSp-7-_QfcRqfAw'
                    response = requests.get(url, headers=headers)  # GET 요청
                    
                    weather_observation_data = response.text
                    test = weather_observation_data.split('\n')[4]
                    test = np.array(test.split(' '))
                    test = test[test != '']
                    data.append(test)
'''

ValueError: list.remove(x): x not in list

In [ ]:
weather_obv_df = pd.DataFrame(data = data, columns = columns)
weather_obv_df.to_csv('경상대_기상_일자관측데이터.csv')

In [52]:
'''
test = weather_observation_data.split('\n')[2]
test = np.array(test.split(' '))
columns = list(test[test != ''])
columns.remove('#')

data = []
#데이터 추출
for i in tqdm(range(4,len(weather_observation_data.split('\n')))):
    text = weather_observation_data.split('\n')[i]
    if(weather_observation_data.split('\n')[i] == '#7777END'):
        break
    text = np.array(text.split(' '))
    text = text[text != '']
    data.append(text)
'''

#START7777
#2345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234
# YYMMDDHHMI STN  WD   WS GST  GST  GST     PA     PS PT    PR    TA    TD    HM    PV     RN     RN     RN     RN     SD     SD     SD WC WP WW                      CA  CA   CH CT        CT  CT  CT    VS   SS    SI ST    TS    TE    TE    TE    TE  ST   WH BF IR IX
#        KST  ID  16  m/s  WD   WS   TM    hPa    hPa  -   hPa     C     C     %   hPa     mm    DAY    JUN    INT    HR3    DAY    TOT -- -- ---------------------- TOT MID  MIN -------- TOP MID LOW                  GD     C     5    10    20    30 SEA    m --  -  -
202206010000 192   0  0.1  -9 -9.0   -9 1006.6 1010.1  2   1.5  16.8  10.1  65.0  12.4   -9.0   -9.0   -9.0   -9.0   -9.0   -9.0   -9.0 -9 -9 -                        8   0   -9 -         -9  -9  -9  

In [27]:
weather_obv_df = pd.DataFrame(columns = columns)

In [28]:
weather_obv_df

,YYMMDDHHMI,STN,WD,WS,GST,GST,GST,PA,PS,PT,...,TS,TE,TE,TE,TE,ST,WH,BF,IR,IX


In [29]:
data = []
columns = ['YYMMDDHHMI', 'STN', 'WD', 'WS', 'GST', 'GST', 'GST', 'PA', 'PS',
       'PT', 'PR', 'TA', 'TD', 'HM', 'PV', 'RN', 'RN', 'RN', 'RN', 'SD',
       'SD', 'SD', 'WC', 'WP', 'WW', 'CA', 'CA', 'CH', 'CT', 'CT', 'CT',
       'CT', 'VS', 'SS', 'SI', 'ST', 'TS', 'TE', 'TE', 'TE', 'TE', 'ST',
       'WH', 'BF', 'IR', 'IX']
year = dict()
year['2022'] = ['31', '28', '31', '30', '31', '30', '31', '31', '30', '31', '30', '31']
year['2023'] = ['31', '28', '31', '30', '31', '30', '31', '31', '30', '31', '30', '31']
year['2024'] = ['31', '29', '31', '30', '31', '30', '31', '31', '30', '31', '30', '31']
month = [f"{i:02}" for i in range(1, 13)]
time = ('0000','2330')
weather_obv_df = pd.DataFrame(columns = columns)




for y in year.keys():
    for m in month:
        for ds,de in [('01','14'),('15',year[y][int(m)-1])]:
            
            url = f'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php?tm1={y}{m}{ds}{time[0]}&tm2={y}{m}{de}{time[1]}&stn=192&help=0&authKey=wgUybypBT-2FMm8qQT_tbQ'
            response = requests.get(url, headers=headers)  # GET 요청
            weather_observation_data = response.text

            data = []
            #데이터 추출
            for i in tqdm(range(4,len(weather_observation_data.split('\n')))):
                text = weather_observation_data.split('\n')[i]
                if(weather_observation_data.split('\n')[i] == '#7777END'):
                    break
                text = np.array(text.split(' '))
                text = text[text != '']
                data.append(text)
            weather_obv_df = pd.concat([weather_obv_df, pd.DataFrame(data = data, columns = columns)], ignore_index=True)

weather_obv_df.to_csv('경상대_일자별관측.csv', index=False)
'''
data = []
#데이터 추출
for i in tqdm(range(4,len(weather_observation_data.split('\n')))):
    text = weather_observation_data.split('\n')[i]
    if(weather_observation_data.split('\n')[i] == '#7777END'):
        break
    text = np.array(text.split(' '))
    text = text[text != '']
    data.append(text)
'''



'''
for i in tqdm(range(54,len(weather_observation_data.split('\n')))):
    text = weather_observation_data.split('\n')[i]
    if(weather_observation_data.split('\n')[i] == '#7777END'):
        break
    text = np.array(text.split(' '))
    text = text[text != '']
    data.append(text)


response = requests.get(url, headers=headers)  # GET 요청
weather_observation_data = response.text
'''



100%|█████████▉| 408/410 [00:00<00:00, 7090.87it/s]


"\nfor i in tqdm(range(54,len(weather_observation_data.split('\n')))):\n    text = weather_observation_data.split('\n')[i]\n    if(weather_observation_data.split('\n')[i] == '#7777END'):\n        break\n    text = np.array(text.split(' '))\n    text = text[text != '']\n    data.append(text)\n\n\nresponse = requests.get(url, headers=headers)  # GET 요청\nweather_observation_data = response.text\n"

In [30]:
weather_obv_df

,YYMMDDHHMI,STN,WD,WS,GST,GST,GST,PA,PS,PT,...,TS,TE,TE,TE,TE,ST,WH,BF,IR,IX
0,202201010000,192,0,0.0,-9,-9.0,-9,1027.3,1031.1,2,...,-1.0,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
1,202201010100,192,16,0.7,-9,-9.0,-9,1027.7,1031.5,-9,...,-1.2,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
2,202201010200,192,0,0.1,-9,-9.0,-9,1027.7,1031.5,-9,...,-1.4,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
3,202201010300,192,0,0.0,-9,-9.0,-9,1027.8,1031.6,2,...,-1.5,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
4,202201010400,192,7,0.7,-9,-9.0,-9,1027.0,1030.8,-9,...,-1.9,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26250,202412311900,192,0,0.2,-9,-9.0,-9,1017.1,1020.8,-9,...,0.0,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,2
26251,202412312000,192,0,0.0,-9,-9.0,-9,1017.9,1021.6,-9,...,-0.7,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,2
26252,202412312100,192,0,0.0,-9,-9.0,-9,1018.5,1022.2,2,...,-1.0,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,2
26253,202412312200,192,0,0.0,-9,-9.0,-9,1018.8,1022.6,-9,...,-1.3,-99.0,-99.0,-99.0,-99.0,-9,-9.0,-9,3,2


In [33]:
len([202210231500, 202210231600, 202210231700, 202210231800, 202210231900, 
 202210232000, 202210232100, 202210232200, 202210232300, 202210240000, 
 202210240100, 202210240200, 202210240300, 202210240400, 202210240500, 
 202210240600, 202210240700, 202210240800, 202210240900, 202210241000, 
 202210241100, 202210241200, 202210241300, 202210241400, 202210241500, 
 202210241600, 202210241700, 202210241800, 202210241900, 202210242000, 
 202210242100, 202210242200, 202210242300, 202210250000, 202210250100, 
 202210250200, 202210250300, 202210250400, 202210250500, 202210250600, 
 202210250700, 202210250800, 202210250900, 202210251000, 202210251100, 
 202210251200, 202210251300, 202210251400, 202210251500])

49

In [35]:
weather_obv_df[weather_obv_df['YYMMDDHHMI'] == 202210231500]

,YYMMDDHHMI,STN,WD,WS,GST,GST,GST,PA,PS,PT,...,TS,TE,TE,TE,TE,ST,WH,BF,IR,IX


In [31]:
weather_obv_df.to_csv('경상대_일자별관측.csv', index=False)

In [36]:
url = f'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php?tm=202210240400&stn=192&help=0&authKey=Brv79B9xSp-7-_QfcRqfAw'
response = requests.get(url, headers=headers)  # GET 요청
weather_observation_data = response.text
print(weather_observation_data)

#START7777
#2345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234
# YYMMDDHHMI STN  WD   WS GST  GST  GST     PA     PS PT    PR    TA    TD    HM    PV     RN     RN     RN     RN     SD     SD     SD WC WP WW                      CA  CA   CH CT        CT  CT  CT    VS   SS    SI ST    TS    TE    TE    TE    TE  ST   WH BF IR IX
#        KST  ID  16  m/s  WD   WS   TM    hPa    hPa  -   hPa     C     C     %   hPa     mm    DAY    JUN    INT    HR3    DAY    TOT -- -- ---------------------- TOT MID  MIN -------- TOP MID LOW                  GD     C     5    10    20    30 SEA    m --  -  -
#7777END



In [37]:
url = f'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php?tm=202310240400&stn=192&help=1&authKey=Brv79B9xSp-7-_QfcRqfAw'
response = requests.get(url, headers=headers)  # GET 요청
weather_observation_data = response.text
print(weather_observation_data)

#START7777
#--------------------------------------------------------------------------------------------------
#  기상청 지상관측 시간자료 [입력인수형태][예] ?tm=201007151200&stn=0&help=1
#--------------------------------------------------------------------------------------------------
#  1. TM     : 관측시각 (KST)
#  2. STN    : 국내 지점번호
#  3. WD     : 풍향 (16방위)
#  4. WS     : 풍속 (m/s)
#  5. GST_WD : 돌풍향 (16방위)
#  6. GST_WS : 돌풍속 (m/s)
#  7. GST_TM : 돌풍속이 관측된 시각 (시분)
#  8. PA     : 현지기압 (hPa)
#  9. PS     : 해면기압 (hPa)
# 10. PT     : 기압변화경향 (Code 0200) 
# 11. PR     : 기압변화량 (hPa)
# 12. TA     : 기온 (C)
# 13. TD     : 이슬점온도 (C)
# 14. HM     : 상대습도 (%)
# 15. PV     : 수증기압 (hPa)
# 16. RN     : 강수량 (mm) : 여름철에는 1시간강수량, 겨울철에는 3시간강수량
# 17. RN_DAY : 일강수량 (mm) : 해당시간까지 관측된 양(통계표)
# 18. RN_JUN : 일강수량 (mm) : 해당시간까지 관측된 양을 전문으로 입력한 값(전문)
# 19. RN_INT : 강수강도 (mm/h) : 관측하는 곳이 별로 없음
# 20. SD_HR3 : 3시간 신적설 (cm) : 3시간 동안 내린 신적설의 높이
# 21. SD_DAY : 일 신적설 (cm) : 00시00분부터 위 관측시간까지 내린 신적설의 높이
# 22. SD_TOT : 적설 (cm) : 치우지 않고 그냥 계

In [ ]:
#weather_obv_df = pd.DataFrame(data = data, columns = columns)

In [ ]:
#일자를 구간으로 설정해서 데이터 불러오는 API인데 작동 잘안됨, 그래서 구간이 아니라 일별로 계속 요청해서 데이터 불러왓음

#저 정보를 전처리해서 판다스데이터프레임 생성, 컬럼정보, 데이터 추출
#컬럼정보 일일이 손으로 치려니 ㅈ같아서 위에 받아온거 데이터 전처리해서 만드는 코드
'''
for i in range(4, 50):
    test = weather_observation_data.split('\n')[i]
    test = np.array(test.split(' '))
    print(test[test != ''])
'''

# 컬럼저 정보를 전처리해서 판다스데이터프레임 생성, 컬럼정보, 데이터 추출
#컬럼정보 일일이 손으로 치려니 ㅈ같아서 위에 받아온거 데이터 전처리해서 만드는 코드
'''
for i in range(4, 50):
    test = weather_observation_data.split('\n')[i]
    test = np.array(test.split(' '))
    print(test[test != ''])
'''

'''
weather_obv_df = pd.DataFrame(columns = columns)


tm1 = ['202201010000','202209010000','202301010000','202309010000','202401010000','202409010000']
tm2 = ['202208312330','202212312330','202308312330','202312312330','202408312330','202412312330']

for i in range(len(tm1)):
    url = f'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php?tm1={tm1[i]}&tm2={tm2[i]}&stn=192&help=1&authKey=Brv79B9xSp-7-_QfcRqfAw'
    response = requests.get(url, headers=headers)  # GET 요청
    weather_observation_data = response.text
    
    # 컬럼추출
    test = weather_observation_data.split('\n')[52]
    test = np.array(test.split(' '))
    columns = list(test[test != ''])
    columns.remove('#')
    print(columns)
    
    data = []
    #데이터 추출
    for i in tqdm(range(54,len(weather_observation_data.split('\n')))):
        text = weather_observation_data.split('\n')[i]
        if(weather_observation_data.split('\n')[i] == '#7777END'):
            break
        text = np.array(text.split(' '))
        text = text[text != '']
        data.append(text)

    temp_df = pd.DataFrame(data = data, columns = columns)

    weather_obv_df = pd.concat([weather_obv_df, temp_df], axis=0, ignore_index=True)
'''

In [ ]:
weather_obv_df